<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/Language_BME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備  


1)ToyoNet-ACEからあらかじめダウンロードしていたテキストファイルを，Google Colabの「ファイル」にドラッグする．  
この講義では次のファイルを用意している．


*   Neko.txt: 夏目漱石「吾輩は猫である」
*   yokaidan.txt: 井上円了「妖怪談」
  
※いずれも[青空文庫](https://www.aozora.gr.jp)より，著作権消滅済み．


2)必要なパッケージをインストールする．MeCabのインストールに少し時間がかかる(約1分)．

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

# N-gram  

"Ngram"は，キーワードの出現頻度を可視化する意味で用いられることが多い．  
[Google Ngram Viewer](https://books.google.com/ngrams) (Google Books)  
[NDL Ngram Viewer](https://lab.ndl.go.jp/ngramviewer/) (国立国会図書館)  

## N-gramをもとにして文章を生成してみる  
参考：[3. Pythonによる自然言語処理　1-1. 単語N-gram](https://qiita.com/y_itoh/items/82222af50bf1f80255eb) @y_itoh(yumi ito)  
  
この方法は純粋に訓練データ(テキスト)の単語の統計情報から文章を生成するものである．

Google Colabにダウンロード済みのテキストファイル(**Neko.txt**)を読み込む．  

読み込んだテキストが表示される．

In [ ]:
with open('Neko.txt', mode='rt', encoding='utf-8') as f:
    read_text = f.read()
nekotxt = read_text

print(nekotxt)

※後で井上円了「妖怪談」にする場合，1行目を次のように修正して再実行すること．  


```
with open('yokaidan.txt')
```

### 分かち書き

MeCabで分かち書きし，結果を表示する．

In [ ]:
import MeCab
tagger = MeCab.Tagger("-Owakati")
nekotxt = tagger.parse(nekotxt)

# print(nekotxt)
nekotxt = nekotxt.split()
print(nekotxt)

### 辞書の作成

N-gram辞書を生成する．  
連続する2語を要素とし，その頻度をカウントしたものがdic2(2-gram)，3語をカウントしたものがdic3(3-gram)である．

In [ ]:
from collections import Counter
import numpy as np
from numpy.random import *

string = nekotxt

# 除外する文字記号
delimiter = ['「', '」', '…', '　']

# 2語のリスト
double = list(zip(string[:-1], string[1:]))
double = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter))), double)

# 3語のリスト
triple = list(zip(string[:-2], string[1:-1], string[2:]))
triple = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter) or (x[2] in delimiter))), triple)

# 要素数をカウントして辞書を生成
dic2 = Counter(double)
dic3 = Counter(triple)

2-gram辞書を表示する．  
長いので中断してしまうが問題ない．  
  
例えば，

```
('聞き', 'たまえ') 3
```
は，「聞き」の直後に「たまえ」があるパターンが3回見つかった，という意味である．


In [ ]:
for u,v in dic2.items():
    print(u, v)

3-gram辞書を表示する．  
これも長いので中断してしまうが問題ない．  
  
同じく，例えば，

```
('飛び', '上っ', 'て') 2
```
は，「飛び」「上っ」「て」の連続パターンが2回見つかった，という意味である．


In [ ]:
for u,v in dic3.items():
    print(u, v)

### 文章生成

文章生成の関数を定義する．

In [ ]:
def nextword(words, dic):
    ## ➀先頭の単語wordsの要素数gramsを取得
    grams = len(words)

    ## ➁N-gram辞書dicから一致する要素を抽出
    # 2語の場合
    if grams == 2:
        matcheditems = np.array(list(filter(
            (lambda x: x[0][0] == words[1]), #1番目が合致
            dic.items())))
    # 3語の場合
    else:
        matcheditems = np.array(list(filter(
            (lambda x: x[0][0] == words[1]) and (lambda x: x[0][1] == words[2]), #1番目と2番目が合致
            dic.items())))

    ## ➂一致する語がない場合のエラーメッセージ
    if(len(matcheditems) == 0):
        print("No matched generator for", words[1])
        return ''

    ## ➃重み付き出現頻度リスト
    # matcheditemsから出現頻度を取得
    probs = [row[1] for row in matcheditems]
    # 0～1の疑似乱数を生成して出現頻度にかける
    weightlist = rand(len(matcheditems)) * probs

    ## ➄matcheditemsから重み付き出現頻度が最大の要素を取得
    if grams == 2:
        u = matcheditems[np.argmax(weightlist)][0][1]
    else:
        u = matcheditems[np.argmax(weightlist)][0][2]
    return u

文章生成を試す．  

1)まずはそのまま実行すると，「吾輩」からはじまる文章が生成される．これは2-gram辞書を用いている．実行するたびに結果が変わるので，何度か試すとよい．  

In [ ]:
# 先頭の単語wordsを入力
words = ['', '吾輩'] # 2-gram
#words = ['', '吾輩', 'は'] # 3-gram

# 出力outputの先頭にwordsを埋め込む
output = words[1:]

# ｢次の語｣を取得
for i in range(100):
    # 2語の場合
    if len(words) == 2:
        newword = nextword(words, dic2)
    # 3語の場合
    else:
        newword = nextword(words, dic3)

    # 出力outputに次の語を追加
    output.append(newword)
    # 次の文字が終止符なら終了
    if newword in ['', '。', '？', '！']:
        break
    # 次のnextwordの準備
    words = output[-len(words):]
    print(words)

# 出力outputを表示
for u in output:
    print(u, end='')


2)次に2行目の冒頭に半角の#を書き加え，逆に3行目の冒頭の#を消去して実行する．今度は3-gram辞書を用いている．これも実行のたびに結果が変わる．  

```
#words = ['', '吾輩'] # 2-gram
words = ['', '吾輩', 'は'] # 3-gram
```
3)最後に好きな単語で文章生成する．たとえば，「吾輩は」の代わりに「猫が」にしてみる．
```
words = ['', '猫', 'が'] # 3-gram
```



  

## 井上円了「妖怪談」を試してみる  
  
「N-gramをもとにして文章を生成してみる」に戻り，**Neko.txt**を**yokaidan.txt**に書き換えて同じように実行してみる．  
最後の文章生成は2-gramで「妖怪」を試してみるとよい(「吾輩」では何も出ない)．  

```
words = ['', '妖怪'] # 2-gram
#words = ['', '吾輩', 'は'] # 3-gram
```



# 文書単語行列と文章類似度  
参考：  
* [scikit-learnのCountVectorizerやTfidfVectorizerの日本語での使い方について](https://qiita.com/kiyuka/items/3de09e313a75248ca029)　@kiyuka
* [Pythonで文書類似度算出！](https://toukei-lab.com/python-mecab)　ウマたん  

## 文書単語行列

次の4つの文章の文書単語行列を求める．あらかじめ分かち書きする．  
* これは最初のドキュメントです。
* このドキュメントは2番目のドキュメントです。
* そして、これは3番目のものです。
* これは最初のドキュメントですか?
  
単に出現頻度が表になっているだけなので難しくはない．

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import display
corpus = [
    ['これ', 'は', '最初', 'の', 'ドキュメント', 'です', '。'],
    ['この', 'ドキュメント', 'は', '2', '番目', 'の', 'ドキュメント', 'です', '。'],
    ['そして', '、', 'これ', 'は', '3', '番目', 'の', 'もの', 'です', '。'],
    ['これ', 'は', '最初', 'の', 'ドキュメント', 'です', 'か', '?']
]
vectorizer = CountVectorizer(analyzer=lambda x: x)

vec = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()
df = pd.DataFrame(vec.toarray(), columns=feature_names)
display(df)

## 二つのURLのテキストの類似度を求める  

### とりあえずやってみる

In [ ]:
import requests
from bs4 import BeautifulSoup
import sys
import MeCab
from time import sleep
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

##関数定義
# Step1：URLからテキスト情報をスクレイピング
def geturl(urls):
    all_text=[]
    for url in urls:
        r=requests.get(url)
        c=r.content
        soup=BeautifulSoup(c,"html.parser")
        article1_content=soup.find_all("p")
        temp=[]
        for con in article1_content:
            out=con.text
            temp.append(out)
        text=''.join(temp)
        all_text.append(text)
        sleep(1)
    return all_text

# Step2：それらをMeCabで形態素解析。名詞だけ抽出。
def mplg(article):
    word_list = ""
    m=MeCab.Tagger()
    m1=m.parse (text)
    for row in m1.split("\n"):
        word =row.split("\t")[0]#タブ区切りになっている１つ目を取り出す。ここには形態素が格納されている
        if word == "EOS":
            break
        else:
            pos = row.split("\t")[1]#タブ区切りになっている2つ目を取り出す。ここには品詞が格納されている
            slice = pos[:2]
            if slice == "名詞":
                word_list = word_list +" "+ word
    return word_list

# Step3：名詞の出現頻度からTF-IDF/COS類似度を算出。テキスト情報のマッチ度を測る
def tfidf(word_list):
    docs = np.array(word_list)#Numpyの配列に変換する
    #単語を配列ベクトル化して、TF-IDFを計算する
    vecs = TfidfVectorizer(
                token_pattern=u'(?u)\\b\\w+\\b'#文字列長が 1 の単語を処理対象に含めることを意味します。
                ).fit_transform(docs)
    vecs = vecs.toarray()
    return vecs


def cossim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

##実装
word_list=[]
texts=geturl(["https://toukei-lab.com/conjoint","https://toukei-lab.com/correspondence"])
for text in texts:
    word_list.append(mplg(text))

vecs=tfidf(word_list)
print(cossim(vecs[1],vecs[0]))

最後に数字が1つ表示されたが，これが次の二つのURL(ソースコードでは59行目付近に記載)のコサイン類似度である．  
**全く同じ文章の場合，1.0**となる．  
* https://toukei-lab.com/conjoint
* https://toukei-lab.com/correspondence

### 名詞の抽出

中身を解説する．MeCabで分かち書きを行うところまでは同じだが，今回は名詞だけを抽出する．結果を見てみる．

In [ ]:
print('最初のURLの分かち書きの結果(名詞のみ): 単語数{}'.format(len(word_list[0])))
print(word_list[0])
print('二番目のURLの分かち書きの結果(名詞のみ): 単語数{}'.format(len(word_list[1])))
print(word_list[1])

### 文書単語行列の表示

文書単語行列を表示する．  
各々の文章の文書ベクトル(tf-idf)の中身を見ると，二つの文章で異なることがわかる．  
もし同じ文章であれば二つのベクトルは全く同じであり，なす角度は0度なので，コサイン類似度cos(0)=1.0となる．

In [ ]:
print('ベクトルのサイズ: {}'.format(len(vecs[0])))
print('\n最初のURLの名詞のtf-idf')
print(vecs[0])
print('\n二番目のURLのの名詞のtf-idf')
print(vecs[1])

idfの計算方法は複数あり，上記のscikit-learnのTfidfVectorizerはデフォルトで次の式となっている．  
$$idf=\ln \frac{すべてのテキスト数+1}{その単語を含むテキスト数+1}+1$$
東大スライドとの違いは，分母分子に＋１があること，対数が自然対数(ネイビア数が底)である．  
これにtf(単語の頻度)を乗じたものがtf-idfであるが，TfidfVectorizerではtf-idfは最終的にはベクトルごとに正規化(二乗和が1)しているので，どれも1未満である．  
  
  参考：  
  * [Scikit learn: 6.2 Feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
  * [Wikipedia: tf-idf](https://ja.wikipedia.org/wiki/Tf-idf)

# 単語の分散表現とword2vec

word2vecは2010年に当時Googleに在籍していたT. Mikolovらにより開発されたシンプルなRNNであり，次のような特徴を持つ．  
* 単語は多次元ベクトルで表現される(これを分散表現と呼ぶ)
* 似た単語は似たベクトルを有する  
  
有名な例として"King - man + woman = queen"が挙げられることが多い．manからkingへのベクトルと，womanからqueenへのベクトルが等しい．  
  
参考：  
* [word2vecをColab環境で使うための5行](https://qiita.com/Ninagawa123/items/6c38160e041b6c333905) @Ninagawa123(Izumi Ninagawa) ※word2vecの旧バージョンなのでそのままでは動かない

### Wikipediaから抽出したコーパスの準備

まず"text8"というコーパスを読み込む．これはWikipediaから抽出した100MBほどの前処理済みデータセットである．    
参考：
* [text8](https://mattmahoney.net/dc/textdata.html)

In [ ]:
import gensim.downloader as gendl
corpus = gendl.load("text8")

### word2vecによる学習

次にword2vecを用いて学習させ，ベクトル化する．しばらく時間がかかる(約7分)．  
[gensim](https://radimrehurek.com/gensim/)は自然言語処理のライブラリ．

In [ ]:
import gensim #ライブラリgensimを導入する
model = gensim.models.Word2Vec(corpus, vector_size=200, window=5, epochs=10, min_count=1)

### 学習結果

では，"coffee"に近い単語を表示してみる．表示される数字は，文章類似度で取り上げたコサイン類似度である(つまり全く同じ単語で1.0)．

In [ ]:
model.wv.most_similar('coffee')

**coffee**の部分を各自で書き換えて実行してみよう．  
ただし，固有名詞ではうまくいかない(例：Japan)．また，コーパスが英語なので日本語は認識しない．

"coffee"のベクトルの中身を見てみる．学習の時にベクトルサイズを200で設定したので，このデータは200次元のベクトルデータとなる．

In [ ]:
print(model.wv['coffee'])

word2vecは"coffee"に一番近い単語としてbananaを挙げたが，そのbananaのベクトルも見てみる．先ほどのcoffeeと似てるだろうか？

In [ ]:
print(model.wv['banana'])

## 単語間の演算  


「女王」＋「男」－「女」を計算してみよう！

In [ ]:
model.wv.most_similar(positive=['queen', 'man'], negative=['woman'])

「コーヒー」＋「牛乳」は？

In [ ]:
model.wv.most_similar(positive=['coffee','milk'])

「ケーキ」＋「塩」－「砂糖」は？

In [ ]:
model.wv.most_similar(positive=['cake', 'salt'], negative=['sugar'])